In [ ]:
# default_exp readuniprot

# readuniprot

> Parse Uniprot/Swissprot text format files (.dat), and Uniref XML files.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import collections
import csv
import itertools
import random
import re

from proteinscan import utils

## Code to parse Uniprot/Swissprot .dat files (text format protein database):

In [ ]:
# export

def iterDat(datFPath) :
    """
    Iterator yielding a list of lines for each entry in the .dat file.
    """
    with utils.openGzipOrText(datFPath) as f :
        for (isEnd,it) in itertools.groupby(f, lambda ln : ln.strip()=='//') :
            if not isEnd :
                yield(list(it))

def datEntryLnsWithCode(datEntry,code) :
    "Returns a list of the lines in a .dat file entry with the given code."
    res = []
    for ln in datEntry :
        ln = ln.strip()
        if ln.startswith(code+' ') :
            res.append(ln[len(code):].lstrip())
    return res

def datEntryPrimaryAC(datEntry) :
    "Returns the primary accession number from a .det file entry."
    acLns = datEntryLnsWithCode(datEntry,'AC')
    return acLns[0].split(';')[0]

In [ ]:
entries = list(iterDat('uniprotTest.dat.gz'))
assert (len(entries)==185
        and len(entries[2])==61
        and entries[2][0]=='ID   002R_IIV3               Reviewed;         458 AA.\n'
        and entries[2][10]=='OX   NCBI_TaxID=345201;\n'
        and entries[2][-1]=='     QSIDRYFCSL DSNYNSEDED FEYDSDSEDD DSDSEDDC\n'
        and datEntryLnsWithCode(entries[2],'DT') ==
            ['16-JUN-2009, integrated into UniProtKB/Swiss-Prot.',
             '11-JUL-2006, sequence version 1.',
             '02-JUN-2021, entry version 28.']
        and datEntryPrimaryAC(entries[2])=='Q197F8')

In [ ]:
# export

def scanDat(datFPath, fn, returnFull=False, **kwargs) :
    """
    Scan a function across each entry in a .dat file, accumulating the results.
    If the function returns None, the entry is ignored; otherwise the result
    is added to the returned results list. Accepts additional keyword arguments,
    which are passed on to the function to be scanned.

    If returnFull is True, also returns a list of the corresponding full entries.
    """
    res = []
    if returnFull : fullRes = []
    for datEntry in iterDat(datFPath) :
        item = fn(datEntry, **kwargs)
        if item is not None :
            res.append(item)
            if returnFull :
                fullRes.append(datEntry)
    return (res,fullRes) if returnFull else res

def allPrimaryACsInDat(datFPath) :
    "Returns a list of all primary accession numbers in a .dat file."
    return scanDat(datFPath,datEntryPrimaryAC)

def datEntryPE(datEntry) :
    "Returns the level of evidence for a protein's existence (1-5, lower is stronger)."
    peLns = datEntryLnsWithCode(datEntry,'PE')
    return int(peLns[0][0])

def datEntryName(datEntry) :
    "Returns a descriptive name from a .dat file entry."
    deLns = datEntryLnsWithCode(datEntry,'DE')
    name = deLns[0].split('=')[1].rstrip(';')
    flagSet = set()
    for deLn in deLns[1:] :
        if deLn.startswith('Flags') :
            flagSet.update(flag.rstrip(';') for flag in deLn.split()[1:])
    if len(flagSet) > 0 :
        name += ' (' + '; '.join(sorted(flagSet)) +')'
    return name

def datEntrySeq(datEntry) :
    "Returns the amino acid sequence from a .dat file entry as a single string."
    seqL = []
    for ln in reversed(datEntry) :
        ln = ln.strip()
        if ln.startswith('SQ ') :
            break
        seqL.append(ln.replace(' ',''))
    return ''.join(reversed(seqL))

In [ ]:
acs = allPrimaryACsInDat('uniprotTest.dat.gz')
pes = scanDat('uniprotTest.dat.gz',datEntryPE)
names = scanDat('uniprotTest.dat.gz',datEntryName)
seqs = scanDat('uniprotTest.dat.gz',datEntrySeq)
assert (len(acs)==185
        and acs[:5]==['Q6GZX4', 'Q6GZX3', 'Q197F8', 'Q197F7', 'Q6GZX2']
        and acs[-5:]==['P0C9G3', 'P0C9G4', 'P0C9G1', 'P18559', 'P0C9G6']
        and len(pes)==185
        and pes[:5]==[4, 4, 4, 4, 3]
        and pes[-5:]==[3, 3, 3, 3, 3]
        and len(names)==185
        and names[:3]==['Putative transcription factor 001R',
                      'Uncharacterized protein 002L', 'Uncharacterized protein 002R']
        and names[-3:]==['Protein MGF 110-1L', 'Protein MGF 110-2L (Precursor)',
                         'Protein MGF 110-2L (Precursor)']
        and len(seqs)==185
        and len(seqs[0])==256
        and seqs[0]=='MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGH'
            +'FSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTK'
            +'QHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDFKALVESAHRMRQGHMINVKYILYQLLKKHGHG'
            +'PDGPDILTVKTGSKGVLYDDSFRKIYTDLGWKFTPL'
        and len(seqs[-1])==113
        and seqs[-1]=='MGFFSYLGLVLVGLASLTSLASLANLQDFSTDNPLEEELRCWCQYVKNCRFCWACQDGFCKNKVLK'
            +'NMPSVQEHSYPMEHCMIHRQCKYVRDGPIFQVECMMQTCDAIHLLNA')

In [ ]:
# export

def datEntryGOLines(datEntry) :
    "Returns the lines with GO terms from the database x-ref section of a .dat file entry."
    drLns = datEntryLnsWithCode(datEntry,'DR')
    return [drLn for drLn in drLns if drLn.startswith('GO;')]

def datEntryGOTermPresent(datEntry,goTerm) :
    "Returns 'pos' or 'neg' accordingly as a .dat file entry includes the given GO term."
    if any(goTerm in goLn for goLn in datEntryGOLines(datEntry)) :
        return 'pos'
    return 'neg'

def datEntryIsAtpBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the ATP binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0005524;')
def datEntryIsGtpBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the GTP binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0005525;')
def datEntryIsMetalBinding(datEntry) :
    "Returns 'pos' or 'neg accordingly as a .dat file entry has the metal binding GO term."
    return datEntryGOTermPresent(datEntry,'GO:0046872;')

def datEntryKWs(datEntry) :
    "Returns a list of the keywords from the KW section of a .dat file entry."
    res = []
    for kwLn in datEntryLnsWithCode(datEntry,'KW') :
        for kw in kwLn.split(';') :
            kw = kw.strip().rstrip('.').strip().lower()
            if kw != '' :
                res.append(kw)
    return res

In [ ]:
goLns = scanDat('uniprotTest.dat.gz',datEntryGOLines)
kws = scanDat('uniprotTest.dat.gz',datEntryKWs)
assert (goLns[0]==['GO; GO:0046782; P:regulation of viral transcription; IEA:InterPro.']
        and goLns[1]==['GO; GO:0033644; C:host cell membrane; IEA:UniProtKB-SubCell.',
                       'GO; GO:0016021; C:integral component of membrane; IEA:UniProtKB-KW.']
        and goLns[-1]==[]
        and datEntryGOTermPresent(entries[0],'GO:0046782')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0033644')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0016021')=='pos'
        and datEntryGOTermPresent(entries[1],'GO:0046782')=='neg'
        and datEntryGOTermPresent(entries[-1],'GO:0046782')=='neg'
        and kws[0]==['activator','reference proteome','transcription',
                     'transcription regulation']
        and kws[1]==['host membrane','membrane','reference proteome',
                     'transmembrane','transmembrane helix']
        and kws[-1]==['early protein', 'signal'])

In [ ]:
# export

aaLetters = 'ACDEFGHIKLMNPQRSTVWY'
aaLettersSet = set(aaLetters)

def listIfStr(lOrStr) :
    return [lOrStr] if isinstance(lOrStr,str) else lOrStr

def filterDatEntry(datEntry, restrictTo20AA=True,
            minLen=50, maxLen=400, maxPE=None,
            requireInSpecies='', elimKWs=[], requireKWs=[],
            requireInName='', excludeStrs=[]) :
    """
    Filters a .dat file entry for inclusion in a LM or classification data set.
    The filtering is based on the keyword arguments.

    Returns (primary_accession_num,sequence) if the entry passes the filter, else None.
    """
    seq = datEntrySeq(datEntry)
    kws = datEntryKWs(datEntry)
    elimKWs = listIfStr(elimKWs)
    requireKWs = listIfStr(requireKWs)
    excludeStrs = listIfStr(excludeStrs)
    if ((restrictTo20AA and any(c not in aaLettersSet for c in seq))
        or any(elimKW in kws for elimKW in elimKWs)
        or any(requireKW not in kws for requireKW in requireKWs)
        or (minLen is not None and len(seq)<minLen)
        or (maxLen is not None and len(seq)>maxLen)
        or (maxPE is not None and datEntryPE(datEntry)>maxPE)
        or requireInSpecies not in datEntryLnsWithCode(datEntry,'OS')[0].lower()) :
        return None
    lName = datEntryName(datEntry).lower()
    if ((requireInName not in lName)
        or any (excludeStr in lName for excludeStr in excludeStrs)) :
        return None
    return (datEntryPrimaryAC(datEntry),seq)

In [ ]:
assert (len(scanDat('uniprotTest.dat.gz',filterDatEntry))==166
        and len(scanDat('uniprotTest.dat.gz',filterDatEntry,maxPE=3))==50
        and len(scanDat('uniprotTest.dat.gz',filterDatEntry,maxPE=3,
                        requireKWs=['transmembrane']))==15)

## Code to parse UniRef files (precalculated clusters of similar protein sequences):

### UniRef XML format

UniRef files contain precomputed clusters of similar protein sequences,
at different thresholds of sequence identity (100, 90, 50 - these are percentages).
UniRef XML files are named, `unirefN.xml.gz` where N is the identity threshold (for example `uniref50.xml.gz`).
These files define a set of clusters, each of which contains a set of member sequences.

Each cluster has two special member sequences (these may be the same) - a **seed** sequence, which is simply the longest
sequence in the cluster, and a **representative** sequence, which is chosen to be the most "informative" sequence
in the cluster using a number of criteria including quality of information available, length, etc.
Cluster identifiers are constructed from the prefix `UniRef`,
followed by the sequence identity threshold (`100`, `90`, or `50`),
followed by underscore and the UniProt accession number or UniParc id of the chosen **representative** sequence for the cluster
(for example, `UniRef50_A0A5A9P0L4` or `UniRef50_UPI0016133188`).

<!-- A sequence in a cluster will always be at least N% identical to the seed sequence of that cluster,
where N is the identity threshold (100, 90, or 50). In the case of UniRef90 and UniRef50,
a sequence will also always be at least 11 AA long and have at least 80% overlap with the seed sequence;
these requirements aren't applied to UniRef100.
 -->

In more detail, the XML format consists of a series of `entry` elements. Each `entry` defines a cluster,
and consists of one `representativeMember` element and a series of additional `member` elements,
with each member identifying one sequence in the cluster.
Each `entry` also has some attributes and elements describing the cluster as a whole,
including an `id` attribute containing the cluster identifier constructed as described above,
a `name` element with a descriptive name,
and some `property` elements with different `type` attributes, including `type="member_count"`
(number of sequences in the cluster), `type="common taxon"`, `type="common taxon ID"`,
and `type="GO Molecular Function"` (I assume these last elements contain
GO terms that are common to all members of the cluster).

The `representativeMember` and `member` elements contain a `dbReference` element
identifying the member sequence. This element includes:
- Attributes `type` specifying a database (for example `UniParc ID` or `UniProtKB ID`) and `id` within that database (for example `UPI000FFEDAD7` or `A0A410P257_9BACT`)
- If the sequence is in UniProt, one or more `property` elements with `type="UniProtKB accession"` giving UniProt accession numbers for the sequence (for example `A0A410P257`) - if more than one, these will all be alternates for the same sequence in UniProt.
- If the top level `id` attribute is from UniProt, a `property` element with `type="UniParc ID"` giving the UniParc id for the sequence.
- For UniRef90 and UniRef50, a `property` element with `type="UniRef100 ID"` and `value=` the UniRef100 cluster id for the sequence.
- For UniRef50, a `property` element with `type="UniRef90 ID"` and `value=` the UniRef90 cluster id for the sequence.
- Some `property` elements with information about the sequence, including `type="length"`, `type="protein name"`, `type="source organism"`, `type="NCBI taxonomy"`

The `representativeMember` elements also contain a `sequence` element giving the actual AA sequence.


In [ ]:
#export 

def parseClustersFromUniref(unirefFPath,clusterFPath,encoding='ISO-8859-1') :
    """
    Reads a UniRef XML file and generates a file just giving the cluster info.
    Each line of the file is a space-separated list of primary accession numbers
    from UniProt, which have been grouped into a cluster in the UniRef file.
    Only keeps the entries in UniRef that have a UniProt accession number.
    """
    nEntries = nAccNos = 0
    with utils.openGzipOrText(unirefFPath,encoding) as f :
        with open(clusterFPath,'w') as outF :
            for i,ln in enumerate(f) :
                if (i%100000000)==0 :  # print every 100-millionth line
                    print(i,ln.strip())
                if '<entry' in ln :  # start a cluster
                    memCount = 0
                    expectedMemCount = None
                    entryAccNos = []
                    nEntries += 1
                elif '</entry' in ln :  # end a cluster
                    if memCount != expectedMemCount :
                        print('member count mismatch',memCount,expectedMemCount)
                    nAccNos += len(entryAccNos)
                    if len(entryAccNos) >= 1 :
                        outF.write(" ".join(entryAccNos)+'\n')
                elif '<member' in ln or '<representativeMember' in ln :  # start a sequence member
                    memAccNos = []
                elif '</member' in ln or '</representativeMember' in ln :  # end a sequence member
                    entryAccNos.extend(memAccNos)
                    memCount += 1
                elif '<property' in ln :  # parse a property line
                    m = re.search(r'<property.*type="(.*?)".*value="(.*?)"',ln)
                    if m is not None :
                        t,v = m.groups()
                        if t.lower()=='member count' :  # member count from entry element
                            expectedMemCount = int(v.strip())
                        elif t.lower()=='uniprotkb accession' : # accession number from member element
                            memAccNos.append(v.strip())
    print(nEntries,'entries',nAccNos,'accNos')

In [ ]:
parseClustersFromUniref('unirefTest.xml.gz','unirefTestClusters.txt')
unirefL = [ln.split() for ln in open('unirefTestClusters.txt').read().splitlines()]
assert ([len(accNoL) for accNoL in unirefL] == [1, 1, 1232, 22, 1, 21]
        and unirefL[0]==['A0A5A9P0L4']
        and unirefL[2][:5]==['Q8WZ42', 'A6NKB1', 'E7EQE6', 'E7ET18', 'K7ENY1']
        and unirefL[2][-5:]==['A0A2D4NV39', 'A0A2D4GXH3', 'A0A6P7QTE9', 'A0A6P7QKP2', 'A0A6P7QQP5']
        and unirefL[-1]==['K7G060', 'M7AMC7', 'G3URQ7', 'G1NB19', 'A0A091K6H9', 'A0A7K4ZDL0',
                          'A0A7L0GLZ9', 'A0A7L0TQB1', 'A0A7L1IDD4', 'G3UQ76', 'G3UR09', 'A0A7K5D0T2',
                          'A0A7K9EPM6', 'A0A091EPU3', 'A0A091EPV7', 'A0A091ESE9', 'A0A7K7BU71',
                          'A0A7L1PTK0', 'A0A7L1UQH9', 'A0A7K7BUC8', 'A0A7L4KN96']
       )

0 <?xml version="1.0" encoding="ISO-8859-1" ?>
8 entries 1278 accNos


In [ ]:
# experiment
assert fastai is not None